In [45]:
import numpy as np
import pandas as pd
import random
from typing import List, Tuple
from itertools import combinations
from geopy.distance import geodesic
POPULATION_SIZE = 100              # Dimensione della popolazione
OFFSPRING_SIZE = 50                # Numero di discendenti per generazione
MAX_GENERATIONS = 1000             # Numero massimo di generazioni
MUTATION_RATE = 0.3                # Probabilità di mutazione

# Caricamento delle città e costruzione della matrice delle distanze
CITIES = pd.read_csv('./italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# Parametri per l'algoritmo genetico
NUM_CITIES = len(CITIES)           # Numero di città



## Fitness

In [46]:
# Funzione per calcolare il costo del percorso (lunghezza totale)
def path_cost(path: List[int]) -> float:
    return sum(DIST_MATRIX[path[i], path[(i + 1) % NUM_CITIES]] for i in range(NUM_CITIES))


# Funzione di fitness: inverso della lunghezza del percorso
def fitness(individual: List[int]) -> float:
    return  -path_cost(individual)

## Inizialization

In [47]:

def KKN(start_city: str) -> Tuple[List[int], float]:
    start_city_index = CITIES[CITIES['name'] == start_city].index[0]
    path = [start_city_index]
    visited = set(path)
    current_city = start_city_index

    while len(visited) < len(CITIES):
        nearest_distance = float('inf')

        # Trova le città più vicine
        for city_index in range(len(CITIES)):
            if city_index not in visited:
                distance = DIST_MATRIX[current_city, city_index]
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_city = city_index

        path.append(nearest_city)
        visited.add(nearest_city)
        current_city = nearest_city

    path.append(start_city_index)  # ritorna alla partenza

    # Ottimizzazione locale: 2-opt
    path = tweak(path)
    
    return path


def tweak(path: List[int]) -> List[int]:
    """ Esegue l'ottimizzazione 2-opt sul percorso dato. """
    improved = True
    while improved:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path)):
                if j - i == 1:  # Evita di invertire due punti consecutivi
                    continue
                new_path = path[:]
                new_path[i:j] = reversed(path[i:j])  # Inverti la sottosequenza
                if path_cost(new_path) < path_cost(path):
                    path = new_path
                    improved = True
    return path


def initialize_population(size: int, start_city: str ) -> List[List[int]]:
    # Trova l'indice della città di partenza
    start_city_index = CITIES[CITIES['name'] == start_city].index[0]
    
    # Lista degli indici delle città escludendo quella di partenza
    other_cities = [i for i in range(NUM_CITIES) if i != start_city_index]
    
    # Genera la popolazione fissando la città di partenza e arrivo
    population = []
    for _ in range(size):
        individual = [start_city_index]  # Messina come primo elemento
        individual += list(np.random.permutation(other_cities))  # Permutazione delle altre città
        individual.append(start_city_index)  # Messina come ultimo elemento
        population.append(individual)
    
    return population


## Parent selection function

In [48]:
# Selezione dei genitori tramite torneo
def parent_selection(population: List[List[int]]) -> List[int]:
    tournament = random.sample(population, 5)
    tournament.sort(key=fitness, reverse=True)
    return tournament[0]

## Crossover functions

In [49]:
def inver_over_crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    # Copia il percorso del primo genitore per iniziare a creare il figlio
    
    # Crea copie dei genitori escludendo la città di partenza e di arrivo
    modified_parent1 = parent1[1:-1]  # Escludi il primo e l'ultimo elemento
    modified_parent2 = parent2[1:-1]  # Escludi il primo e l'ultimo elemento

    child = modified_parent1[:]
    # Seleziona una città casuale (gene) all'interno del percorso, esclusa la partenza e arrivo
    gene_index = random.randint(0, len(modified_parent1) - 1)
    gene = child[gene_index]
    
  # Trova la posizione della città successiva nel secondo genitore
    next_index = (modified_parent2.index(gene) + 1) % len(modified_parent2)
    next_city = modified_parent2[next_index]
    
    '''while next_city not in child or next_city == start_city:
        next_index = (next_index + 1) % len(parent2)
        next_city = parent2[next_index]'''

    next_index = child.index(next_city)

    # Esegue l'inversione della sottosequenza tra gene_index e next_index
    if gene_index < next_index:
        child[gene_index:next_index+1] = reversed(child[gene_index:next_index + 1])
    else:
        child[next_index:gene_index+1] = reversed(child[next_index:gene_index + 1])

    #ora bisogna aggiungere le città in come primo e ultimo elemento parent1[0] e parent1[-1]
    child.insert(0, parent1[0])
    child.append(parent1[0])
    
    
    return child


def inver_over_crossover1(parent1: List[int], parent2: List[int]) -> List[int]:
    # Crea copie dei genitori escludendo la città di partenza e di arrivo
    modified_parent1 = parent1[1:-1]  # Escludi il primo e l'ultimo elemento
    modified_parent2 = parent2[1:-1]  # Escludi il primo e l'ultimo elemento

    # Inizializza il figlio con la città di partenza
    child = [parent1[0]]  # Città di partenza

    # Seleziona un gene casuale (città) da modified_parent1
    gene_index = random.randint(0, len(modified_parent1) - 1)
    gene = modified_parent1[gene_index]

    # Trova la posizione della città successiva nel secondo genitore
    next_index = (modified_parent2.index(gene) + 1) % len(modified_parent2)
    next_city = modified_parent2[next_index]

    # Assicurati che la città successiva non sia già presente nel figlio
    while next_city in child or next_city == parent1[0]:  # Esclude città di partenza
        next_index = (next_index + 1) % len(modified_parent2)
        next_city = modified_parent2[next_index]

    # Se il gene selezionato nel primo genitore non è collegato al prossimo nel secondo genitore, inverte
    if next_city not in child:
        next_index_in_parent1 = modified_parent1.index(gene)
        # Esegui inversione tra gene_index e next_index_in_parent1
        if gene_index < next_index_in_parent1:
            child.extend(reversed(modified_parent1[gene_index:next_index_in_parent1 + 1]))
        else:
            child.extend(reversed(modified_parent1[next_index_in_parent1:gene_index + 1]))
    
    # Aggiungi tutte le altre città che non sono già nel figlio
    for city in modified_parent1:
        if city not in child:
            child.append(city)

    # Aggiungi la città di arrivo (che è anche la città di partenza)
    child.append(parent1[0])

    return child




# Operatore di crossover: Ordinal Crossover (OX)
def crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    # Imposta il primo e l'ultimo elemento fisso (città di partenza/arrivo)
    start, end = sorted(random.sample(range(1, NUM_CITIES - 1), 2))
    child = [parent1[0]] + [None] * (NUM_CITIES - 2) + [parent1[0]]
    
    # Copia il segmento selezionato dal primo genitore
    child[start:end] = parent1[start:end]
    
    # Completa il percorso con città dal secondo genitore, evitando duplicati
    position = end
    for city in parent2:
        if city not in child:
            if position >= NUM_CITIES - 1:
                position = 1  # Riparte dal secondo elemento per evitare il primo e l'ultimo
            child[position] = city
            position += 1

    return child



## Mutation functions

In [50]:
# Operatore di mutazione: Scambia due città casualmente
def mutate(individual: List[int]) -> List[int]:
    if random.random() < MUTATION_RATE:
        # Seleziona due città da scambiare, escludendo la città di partenza e arrivo
        i, j = random.sample(range(1, NUM_CITIES - 1), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual


def inversion_mutation(individual: List[int]) -> List[int]:
    # Assicura che la città di partenza e arrivo sia fissa
    if random.random() < MUTATION_RATE:
        start_city = individual[0]
        end_city = individual[-1]
        
        # Seleziona due indici casuali tra le città interne (1 a len(individual) - 2)
        idx1, idx2 = sorted(random.sample(range(1, len(individual) - 1), 2))
        
        # Inverte la sequenza tra idx1 e idx2
        individual[idx1:idx2 + 1] = reversed(individual[idx1:idx2 + 1])
        
        # Ripristina la città di partenza e arrivo
        individual[0] = start_city
        individual[-1] = end_city
    
    return individual


In [51]:

# Algoritmo genetico principale
population = initialize_population(POPULATION_SIZE, start_city="Syracuse")
population2 = KKN(start_city="Syracuse")
for generation in range(MAX_GENERATIONS):
    # Creazione della prole
    offspring = []
    for _ in range(OFFSPRING_SIZE):
        parent1 = parent_selection(population)
        parent2 = parent_selection(population)
        child = inver_over_crossover(parent1, parent2)
        child = inversion_mutation(child)
        offspring.append(child)
    
    # Valutazione fitness della prole
    population.extend(offspring)
    population.sort(key=fitness, reverse=True)
    population = population[:POPULATION_SIZE]  # Mantiene i migliori

    # Stampa ogni 100 generazioni
    if generation % 100 == 0:
        print(f"Generation {generation}, Best Path Cost: {path_cost(population[0]):.2f} km")

# Risultato finale
best_individual = population[0]
print("Miglior percorso trovato:", [CITIES['name'][i] for i in best_individual])
print("Costo del miglior percorso:", path_cost(best_individual), "km")



Generation 0, Best Path Cost: 13923.93 km


IndexError: list index out of range